## Second Order: Nonlinear Factors in Bar Attributes

This example demonstrates how the **local stiffness matrix** and **load vector** of a bar element change when **nonlinear geometric effects** (Second-Order Theory) are considered.
It serves as an extension of the previous example *["Second Order: Cantilever Colum"](second_order_cantilever.ipynb)*.
For details on solving a complete Second-Order system using the matrix-based approach, please refer to that example.
## Define System
The same cantilever system from the previous example is used here.
It consists of a 4 m vertical cantilever column modeled with one bar, a point load at the free end, and a trapezoidal distributed load along its length.

We first define the material, cross-section, nodes, bar, and assemble them into a `System`.

In [1]:
# Import Modules
from sstatics.core.preprocessing import Bar, BarSecond, BarLineLoad, CrossSection, Material, Node, NodePointLoad, System
from sstatics.core.calc_methods import FirstOrder, SecondOrder

# Setting print options
import numpy as np
np.set_printoptions(precision=3, suppress=True)

# Define cross-section and material
c_1 = CrossSection(0.00002769, 0.007684, 0.2, 0.2, 0.6275377)
m_1 = Material(210000000, 0.1, 81000000, 0.1)

# Define nodes
node_1 = Node(x=0, z=0, u='fixed', w='fixed', phi='fixed')
node_2 = Node(x=0, z=-4, loads=NodePointLoad(x=0, z=182, phi=0, rotation=0))

# Define bar with trapezoidal line load
bar_1 = Bar(
    node_1,
    node_2,
    c_1,
    m_1,
    line_loads=BarLineLoad(pi=1, pj=1.5, direction='z', coord='bar', length='exact')
)

# Create system
system = System([bar_1])

## Compute the Longitudinal Force
To include Second-Order effects, the average longitudinal force (f_axial) in each bar must be known.
This value is obtained from the initial system solution and represents the mean axial force along the member.

For the current example (a single-bar system), only one axial force is returned. We select the first value from the list.

In [2]:
# Compute average longitudinal force for the bar(s)
f_axial = SecondOrder(system).averaged_longitudinal_force[0]

## Define the Second-Order Bar

A bar considering Second-Order effects is defined similarly to a standard Bar,
but with two key differences:

1. The approach (solver method) must be specified, e.g. 'analytic'.

2. The average longitudinal force (f_axial) must be provided.

This allows the bar’s stiffness matrix and load vector to include geometric nonlinearity.

In [3]:
bar_1_second = BarSecond(node_1, node_2, c_1, m_1, line_loads=BarLineLoad(pi=1, pj=1.5, direction='z', coord='bar', length='exact'), approach='analytic', f_axial=f_axial)

## Comparison of Linear and Nonlinear Behavior
Now we can compare the stiffness matrices and load vectors between:

- The First-Order (linear) bar
- The Second-Order (analytic) bar

This illustrates how geometric nonlinearity modifies the element behavior.

In [4]:
# Compare stiffness matrices
print("Stiffness matrix (First Order):\n", bar_1.stiffness_matrix())
print("Stiffness matrix (Second Order - Analytic):\n", bar_1_second.stiffness_matrix())

Stiffness matrix (First Order):
 [[ 1.09029375e+03 -2.46349770e-11 -2.18058750e+03 -1.09029375e+03
   2.46349770e-11 -2.18058750e+03]
 [-2.46349770e-11  4.03410000e+05 -1.33522475e-13  2.46349770e-11
  -4.03410000e+05 -1.33522475e-13]
 [-2.18058750e+03 -1.33522475e-13  5.81490000e+03  2.18058750e+03
   1.33522475e-13  2.90745000e+03]
 [-1.09029375e+03  2.46349770e-11  2.18058750e+03  1.09029375e+03
  -2.46349770e-11  2.18058750e+03]
 [ 2.46349770e-11 -4.03410000e+05  1.33522475e-13 -2.46349770e-11
   4.03410000e+05  1.33522475e-13]
 [-2.18058750e+03 -1.33522475e-13  2.90745000e+03  2.18058750e+03
   1.33522475e-13  5.81490000e+03]]
Stiffness matrix (Second Order - Analytic):
 [[ 1.03485736e+03 -2.46383715e-11 -2.16064695e+03 -1.03485736e+03
   2.46383715e-11 -2.16064695e+03]
 [-2.46383715e-11  4.03410000e+05 -1.32301469e-13  2.46383715e-11
  -4.03410000e+05 -1.32301469e-13]
 [-2.16064695e+03 -1.32301469e-13  5.71379436e+03  2.16064695e+03
   1.32301469e-13  2.92879345e+03]
 [-1.0348573

In [5]:
# Compare load vectors
print("Load vector (First Order):\n", bar_1.f0())
print("Load vector (Second Order - Analytic):\n", bar_1_second.f0())

Load vector (First Order):
 [[-2.30002498e+00]
 [-1.40835912e-16]
 [ 1.60004996e+00]
 [-2.69997502e+00]
 [-1.65325788e-16]
 [-1.73328337e+00]]
Load vector (Second Order - Analytic):
 [[-2.29990391e+00]
 [-1.40828498e-16]
 [ 1.61393947e+00]
 [-2.70009609e+00]
 [-1.65333202e-16]
 [-1.74765718e+00]]


## Summary
The stiffness matrix of the Second-Order bar includes additional terms dependent on the axial force (`f_axial`).
This example highlights the influence of geometric nonlinearity on element-level quantities such as stiffness matrices and load vectors.
It provides a detailed insight into how the `BarSecond` class modifies the bar attributes for Second-Order effects.
For a full system-level Second-Order analysis, refer to the example:
*["Second Order: Cantilever Colum"](second_order_cantilever.ipynb)*.